In [5]:
import numpy as np
import pandas as pd 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
#from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
#import chardet

In [10]:

df = pd.read_csv('C:/Users/raush/OneDrive/Desktop/data/spam.csv',encoding= 'latin')


In [11]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [12]:
df["label"]=df["v1"]
df["message"]=df["v2"]
df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4", "v1", "v2"],axis=1,inplace=True)

In [13]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [14]:
max_features = 1000000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['message'].values)
X = tokenizer.texts_to_sequences(df['message'].values)
X = pad_sequences(X)

In [18]:
from tensorflow import keras
model = keras.Sequential()
model.add(Embedding(max_features, 128, input_length = X.shape[1], trainable=True))
model.add(LSTM(100))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 189, 128)          128000000 
_________________________________________________________________
lstm (LSTM)                  (None, 100)               91600     
_________________________________________________________________
dense (Dense)                (None, 2)                 202       
Total params: 128,091,802
Trainable params: 128,091,802
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
Y = pd.get_dummies(df['label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2) #split dataset to 80% training, 20% testing
print("X_train Shape: ",X_train.shape)
print("Y_train Shape: ",Y_train.shape)
print("X_test Shape: ",X_test.shape)
print("Y_test Shape: ",Y_test.shape)

X_train Shape:  (4457, 189)
Y_train Shape:  (4457, 2)
X_test Shape:  (1115, 189)
Y_test Shape:  (1115, 2)


In [20]:
batch_size = 32
history=model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs = 2, batch_size=batch_size)

Epoch 1/2
140/140 [==============================] - 598s 4s/step - loss: 0.1428 - accuracy: 0.9522 - val_loss: 0.0422 - val_accuracy: 0.9901
Epoch 2/2
140/140 [==============================] - 665s 5s/step - loss: 0.0198 - accuracy: 0.9942 - val_loss: 0.0540 - val_accuracy: 0.9857


In [21]:
def predict_message(message):
    seq = tokenizer.texts_to_sequences(message)

    padded = pad_sequences(seq, maxlen=X.shape[1])

    pred = model.predict(padded)

    labels = ['not spam','spam']
    print(labels[np.argmax(pred)])

In [22]:
message = ['Congratulations! you have won a $1,000 Walmart gift card. Go to http://bit.ly/123456 to claim now.']
predict_message(message)

spam


In [23]:
message = ['thanks for accepting my request to connect']
predict_message(message)

not spam


In [24]:
message = ['I updated the code. Can you check it from this link kaggle.com']
predict_message(message)

not spam


In [25]:
message = ['Congratz you won a 100 dollar check. Click here to claim your money']
predict_message(message)

spam


In [26]:
model.save('spam_sms.h5')